Código responsável por instalar o ambiente necessário para o desenvolvimento do restante do programa

In [ ]:
!pip install python-Levenshtein
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!apt-get update
!google-chrome-stable --version
!apt -y update
!apt install -y wget curl unzip
!wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
!dpkg -i libu2f-udev_1.1.4-1_all.deb
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
!wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
!unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver
!mv /tmp/chromedriver /usr/lib/chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 114.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archiv

Importação das bibliotecas necessárias

In [ ]:
import time
from selenium import webdriver
from time import sleep
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import pandas as pd
import numpy as np
import os
from google.colab import drive
from google.colab import files
import Levenshtein
from selenium.webdriver.chrome.service import Service

Configuração do driver

In [ ]:
try:

  options = webdriver.ChromeOptions()
  options.add_argument('--headless')  # Executar o Chrome em modo headless (sem interface gráfica)
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  service = Service('/usr/lib/chromedriver')
  driver = webdriver.Chrome(service=service, options=options)

  #drive.mount('/content/drive')
  wait = WebDriverWait(driver, 20)

except OSError as e:

  print(e.args)
  print(type(e))
  print(e)
  print(e.with_traceback)

Código de extração dos dados da população

In [ ]:
try:

  cidades = []
  colunas_ibge = ['Sigla UF', 'Nome UF','População Total em 2010','População Urbana em 2010','População Rural em 2010']

  site = "https://www.ibge.gov.br/censo2010/apps/sinopse/index.php"
  driver.get(site)
  driver.implicitly_wait(20)
  siglas = ['RO','AC','AM','RR','PA','AP','TO','MA','PI','CE','RN','PB','PE','AL','SE','BA','MG','ES','RJ','SP','PR','SC','RS','MS','MT','GO','DF']

  opcaoUF = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/form/select[1]/option[2]'))).click()
  time.sleep(2)
  opcaoDados = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/form/select[2]/option[12]'))).click()
  time.sleep(2)
  for i in range(1,28):
    #//*[@id="div_tabela_dados"]/table/tbody/tr[27]/td[2]
    #2 nomeuf
    #3 Pop total
    #4 total homens
    #5 total mulheres
    #6 total urbana
    #7 urbana homens
    #8 urbana mulheres
    #9 rural total
    #10 rural homens
    #11 rural mulheres

    cidades.append(siglas[i-1])
    nomeuf = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="div_tabela_dados"]/table/tbody/tr[%d]/td[2]' % i))).text
    cidades.append(nomeuf)

    populacaototal = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="div_tabela_dados"]/table/tbody/tr[%d]/td[3]' % i))).text
    populacaototal = populacaototal.replace('.','')
    cidades.append(populacaototal)

    populacaourbana = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="div_tabela_dados"]/table/tbody/tr[%d]/td[6]' % i))).text
    populacaourbana = populacaourbana.replace('.','')
    cidades.append(populacaourbana)

    populacaorural = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="div_tabela_dados"]/table/tbody/tr[%d]/td[9]' % i))).text
    populacaorural = populacaorural.replace('.','')
    cidades.append(populacaorural)

  cidades_array = np.array(cidades).reshape(-1, 5)
  df = pd.DataFrame(cidades_array, columns=colunas_ibge)
  with pd.option_context('display.max_rows', None,
      'display.max_columns', None):
    df.to_csv('cidades.csv', sep =';', index = False, encoding="utf-8")
    files.download('cidades.csv')

except OSError as e:

  print(e.args)
  print(type(e))
  print(e)
  print(e.with_traceback)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Código de extração dos dados da tabela 6954 SIDRA


In [ ]:
try:

  colunas_sidra = ['Produto','Nome UF','qtd Estabelecimentos','qtd Estabelecimentos AGFNão','qtd Estabelecimentos AGFSim','qtd Produzida',
              'qtd Produzida AGFNao','qtd Produzida AGFSim', 'Unidade de Medida', 'qtd Vendida','qtd Vendida AGFNao','qtd Vendida AGFSim',
              'R$ Produção Total','R$ Produção AGFNao','R$ Produção AGFSim','R$ Venda Total','R$ Venda AGFNao','R$ Venda AGFSim']
  info = []

  #Constantes
  quantidade_produtos = 57
  quantidade_total = quantidade_produtos*19

  for estado in range(0, 27):

    driver.get("https://sidra.ibge.gov.br/tabela/6954")
    driver.implicitly_wait(10)
    #Origem: //*[@id="dim-C220"] bloco grupo de área total
    origem = wait.until(EC.visibility_of_element_located((By.ID, 'dim-C220')))
    #Destino: //*[@id="dim-V"] bloco variável
    destino = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="editor-layout"]/tbody/tr[1]/th')))
    acao = ActionChains(driver)
    time.sleep(1)
    #Grupo de área total
    acao.drag_and_drop(origem, destino).perform()
    time.sleep(1)
    #Origem: //*[@id="dim-C228"] Produtos
    origem = wait.until(EC.visibility_of_element_located((By.ID, 'dim-C228')))
    destino = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="editor-layout"]/tbody/tr[7]')))
    acao = ActionChains(driver)
    time.sleep(2)
    #Produtos
    acao.drag_and_drop(origem, destino).perform()
    time.sleep(1)
    #Origem: //*[@id="dim-T"] Unidade territorial
    origem = wait.until(EC.visibility_of_element_located((By.ID, 'dim-T')))
    #Destino: //*[@id="dim-V"] bloco variável
    destino = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="editor-layout"]/tbody/tr[1]/th')))
    acao = ActionChains(driver)
    time.sleep(1)
    #Unidade territorial
    acao.drag_and_drop(origem, destino).perform()
    time.sleep(1)

    if estado == 0:
      #Marca os dois botões de Tipologia
      tipAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="panel-C829-collapse"]/div[2]/div/div[2]/div/div/div/div/div[2]/div/div/div/button'))).click()
      tipAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="panel-C829-collapse"]/div[2]/div/div[2]/div/div/div/div/div[3]/div/div/div/button'))).click()
      #Desmarca o Brasil
      buttonBrasil = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="arvore-1025e-1"]/div/div/div/button'))).click()

    #Marcar todas as variáveis
    buttonVariaveis = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="panel-V-collapse"]/div[2]/div/div[1]/div[1]/div/button[1]'))).click()

    #Marcar todos os produtos
    buttonProdutos = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="panel-C228-collapse"]/div[2]/div/div[1]/div[1]/div/button[1]'))).click()

    #Clica no botão para exibir os municípios
    botaoEstado = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="arvore-1105e-1"]/div/div'))).click()

    botaoDesmarcaEstado = wait.until(EC.element_to_be_clickable(
        (By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[1]/div[1]/div/button[2]'))).click()

    a = ActionChains(driver)
    iconeLupa = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[1]/div[2]/div[1]/span')))
    a.move_to_element(iconeLupa).perform()
    time.sleep(2)

    #Campo de inserção do nome do munícipio
    inputNomeEstado = wait.until(EC.visibility_of_element_located((
        By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[1]/div[2]/div[2]/div/input'
    )))
    inputNomeEstado.send_keys(cidades_array[estado][1])
    print(cidades_array[estado][1])
    time.sleep(3)
    #//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/div[1]/div/div/div/button

    buttonEstado = wait.until(EC.visibility_of_element_located((
        By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/div[1]/div/div/div/button'
    )))
    buttonEstado.click()

    time.sleep(2)

    buttonVisualizar = wait.until(EC.element_to_be_clickable((By.ID, 'botaoOk'))).click()
    time.sleep(6)

    #driver.save_screenshot('teste.png')
    #files.download('teste.png')

    for i in range(2, quantidade_produtos+2):

      produto = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/th/span' % i))).text
      info.append(produto)

      info.append(cidades_array[estado][1])

      #Tabela 1 = Variável 1, Número de estabelecimentos
      numEstabelecimentoTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(numEstabelecimentoTotal)

      numEstabelecimentoAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(numEstabelecimentoAGFNao)

      numEstabelecimentoAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(numEstabelecimentoAGFSim)

      #Tabela 2 = Variável 2, Quantidade Produzida
      qtdProduzidaTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(qtdProduzidaTotal)

      qtdProduzidaAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(qtdProduzidaAGFNao)

      qtdProduzidaAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(qtdProduzidaAGFSim)

      #Unidade de Medida
      unidade = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/th/span/span' % (i+1)))).text
      unidade = unidade.replace("(","")
      unidade = unidade.replace(")","")
      info.append(unidade)

      #Tabela 3 = Variável 3, Quantidade Vendida
      qtdVendidaTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[3]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(qtdVendidaTotal)

      qtdVendidaAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[3]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(qtdVendidaAGFNao)

      qtdVendidaAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[3]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(qtdVendidaAGFSim)

      #Tabela 4 = Variável 4, Valor Produção
      valorProducaoT = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[4]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(valorProducaoT)

      valorProducaoAGFN = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[4]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(valorProducaoAGFN)

      valorProducaoAGFS = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[4]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(valorProducaoAGFS)

      #Tabela 5 = Variável 5, Valor Venda
      valorVendaT = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[5]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(valorVendaT)

      valorVendaAGFN = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[5]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(valorVendaAGFN)

      valorVendaAGFS = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[5]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(valorVendaAGFS)

    time.sleep(1)

  info_array = np.array(info).reshape(-1, 18)
  df = pd.DataFrame(info_array, columns=colunas_sidra)
  with pd.option_context('display.max_rows', None,
      'display.max_columns', None):
    df.to_csv('info_br.csv', sep =';', index = False, encoding="utf-8")
    files.download('info_br.csv')

except OSError as e:

  print(e.args)
  print(type(e))
  print(e)
  print(e.with_traceback)

Rondônia
Acre
Amazonas
Roraima
Pará
Amapá
Tocantins
Maranhão
Piauí
Ceará
Rio Grande do Norte
Paraíba
Pernambuco
Alagoas
Sergipe
Bahia
Minas Gerais
Espírito Santo
Rio de Janeiro
São Paulo
Paraná
Santa Catarina
Rio Grande do Sul
Mato Grosso do Sul
Mato Grosso
Goiás
Distrito Federal


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Código de extração dos dados da tabela 6956 SIDRA

In [ ]:
try:

  colunas_sidra_6956 = ['Produto','Nome UF','qtd Estabelecimentos com 50 pés e mais existentes',
    'qtd Estabelecimentos com 50 pés e mais existentes AGFNão','qtd Estabelecimentos com 50 pés e mais existentes AGFSim',
    'qtd Produzida com 50 pés e mais','qtd Produzida com 50 pés e mais AGFNão','qtd Produzida com 50 pés e mais AGFSim', 'Unidade de Medida',
    'qtd Vendida com 50 pés e mais','qtd Vendida AGFNão com 50 pés e mais','qtd Vendida AGFSim com 50 pés e mais',
    'R$ Produção Total das lavouras permanentes com 50 pés e mais','R$ Produção AGFNão com 50 pés e mais','R$ Produção AGFSim com 50 pés e mais',
    'R$ Venda Total com 50 pés e mais','R$ Venda AGFNão das lavouras permanentes com 50 pés e mais','R$ Venda AGFSim com 50 pés e mais',
    'qtd de pés colhidos nos estabelecimentos com 50 pés e mais',
    'qtd de pés colhidos nos estabelecimentos com 50 pés e mais AGFNão','qtd de pés colhidos nos estabelecimentos com 50 pés e mais AGFSim',
    'Área colhida com 50 pés e mais','Área colhida com 50 pés e mais AGFNão','Área colhida com 50 pés e mais AGFSim',
    'Área total existente na data de referência nas lavouras permanentes com 50 pés e mais',
    'Área total existente na data de referência nas lavouras permanentes com 50 pés e mais AGFNão',
    'Área total existente na data de referência nas lavouras permanentes com 50 pés e mais AGFSim',
    'qtd pés nos estabelecimentos com 50 pés e mais','qtd pés nos estabelecimentos com 50 pés e mais AGFNão',
    'qtd pés nos estabelecimentos com 50 pés e mais AGFSim','qtd Estabelecimentos com menos de 50 pés','qtd Estabelecimentos com menos de 50 pés AGFNão',
    'qtd Estabelecimentos com menos de 50 pés AGFSim','qtd pés nos estabelecimentos com menos de 50 pés',
    'qtd pés nos estabelecimentos com menos de 50 pés AGFNão','qtd pés nos estabelecimentos com menos de 50 pés AGFSim',
    'Valor da venda nos estabelecimentos com menos de 50 pés', 'Valor da venda nos estabelecimentos com menos de 50 pés AGFNão',
    'Valor da venda nos estabelecimentos com menos de 50 pés AGFSim', 'Sigla UF', 'Nome UF','População Total em 2010','População Urbana em 2010',
    'População Rural em 2010']
  info = []

  #Constantes
  quantidade_produtos = 61
  quantidade_total = quantidade_produtos*19

  for estado in range(0, 27):

    driver.get("https://sidra.ibge.gov.br/tabela/6956")
    driver.implicitly_wait(10)
    #Origem: //*[@id="dim-C220"] bloco grupo de área total
    origem = wait.until(EC.visibility_of_element_located((By.ID, 'dim-C220')))
    #Destino: //*[@id="dim-V"] bloco variável
    destino = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="editor-layout"]/tbody/tr[1]/th')))
    acao = ActionChains(driver)
    time.sleep(1)
    #Grupo de área total
    acao.drag_and_drop(origem, destino).perform()
    time.sleep(1)
    #Origem: //*[@id="dim-C228"] Produtos
    origem = wait.until(EC.visibility_of_element_located((By.ID, 'dim-C227')))
    destino = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="editor-layout"]/tbody/tr[7]')))
    acao = ActionChains(driver)
    time.sleep(2)
    #Produtos
    acao.drag_and_drop(origem, destino).perform()
    time.sleep(1)
    #Origem: //*[@id="dim-T"] Unidade territorial
    origem = wait.until(EC.visibility_of_element_located((By.ID, 'dim-T')))
    #Destino: //*[@id="dim-V"] bloco variável
    destino = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="editor-layout"]/tbody/tr[1]/th')))
    acao = ActionChains(driver)
    time.sleep(1)
    #Unidade territorial
    acao.drag_and_drop(origem, destino).perform()
    time.sleep(1)

    if estado == 0:
      #Marca os dois botões de Tipologia
      tipAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="panel-C829-collapse"]/div[2]/div/div[2]/div/div/div/div/div[2]/div/div/div/button'))).click()
      tipAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="panel-C829-collapse"]/div[2]/div/div[2]/div/div/div/div/div[3]/div/div/div/button'))).click()
      #Desmarca o Brasil
      buttonBrasil = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="arvore-1195e-1"]/div/div/div/button'))).click()

    #Marcar todas as variáveis
    buttonVariaveis = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="panel-V-collapse"]/div[2]/div/div[1]/div[1]/div/button[1]'))).click()

    #Marcar todos os produtos
    buttonProdutos = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="panel-C227-collapse"]/div[2]/div/div[1]/div[1]/div/button[1]'))).click()

    #Clica no botão para exibir os municípios
    botaoEstado = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="arvore-1275e-1"]/div/div/div'))).click()

    botaoDesmarcaEstado = wait.until(EC.element_to_be_clickable(
        (By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[1]/div[1]/div/button[2]'))).click()

    a = ActionChains(driver)
    iconeLupa = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[1]/div[2]/div[1]/span')))
    a.move_to_element(iconeLupa).perform()
    time.sleep(2)

    #Campo de inserção do nome do munícipio
    inputNomeEstado = wait.until(EC.visibility_of_element_located((
        By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[1]/div[2]/div[2]/div/input'
    )))
    inputNomeEstado.send_keys(cidades_array[estado][1])
    print(cidades_array[estado][1])
    time.sleep(3)
    #//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/div[1]/div/div/div/button

    #driver.save_screenshot('teste.png')
    #files.download('teste.png')

    buttonEstado = wait.until(EC.visibility_of_element_located((
      By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/div[1]/div/div/div/button'
    )))
    buttonEstado.click()

    time.sleep(2)

    buttonVisualizar = wait.until(EC.element_to_be_clickable((By.ID, 'botaoOk'))).click()
    time.sleep(8)

    for i in range(2, quantidade_produtos+2):

      #driver.save_screenshot('teste.png')
      #files.download('teste.png')
      produto = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/th/span' % i))).text
      info.append(produto)

      info.append(cidades_array[estado][1])

      #Tabela 1 = Variável 1, Número de estabelecimentos
      numEstabelecimentoTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(numEstabelecimentoTotal)

      numEstabelecimentoAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(numEstabelecimentoAGFNao)

      numEstabelecimentoAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(numEstabelecimentoAGFSim)

      #Tabela 2 = Variável 2, Quantidade Produzida
      qtdProduzidaTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(qtdProduzidaTotal)

      qtdProduzidaAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(qtdProduzidaAGFNao)

      qtdProduzidaAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(qtdProduzidaAGFSim)

      #Unidade de Medida
      unidade = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/th/span/span' % (i+1)))).text
      unidade = unidade.replace("(","")
      unidade = unidade.replace(")","")
      info.append(unidade)

      #Tabela 3 = Variável 3, Quantidade Vendida
      qtdVendidaTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[3]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(qtdVendidaTotal)

      qtdVendidaAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[3]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(qtdVendidaAGFNao)

      qtdVendidaAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[3]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(qtdVendidaAGFSim)

      #Tabela 4 = Variável 4, Valor Produção
      valorProducaoT = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[4]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(valorProducaoT)

      valorProducaoAGFN = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[4]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(valorProducaoAGFN)

      valorProducaoAGFS = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[4]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(valorProducaoAGFS)

      #Tabela 5 = Variável 5, Valor Venda
      valorVendaT = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[5]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(valorVendaT)

      valorVendaAGFN = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[5]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(valorVendaAGFN)

      valorVendaAGFS = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[5]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(valorVendaAGFS)

      #Tabela 6 = Variável 6,
      numEstabelecimentoTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[6]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(numEstabelecimentoTotal)

      numEstabelecimentoAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[6]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(numEstabelecimentoAGFNao)

      numEstabelecimentoAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[6]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(numEstabelecimentoAGFSim)

      #Tabela 7 = Variável 7,
      qtdProduzidaTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[7]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(qtdProduzidaTotal)

      qtdProduzidaAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[7]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(qtdProduzidaAGFNao)

      qtdProduzidaAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[7]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(qtdProduzidaAGFSim)

      #Tabela 8 = Variável 8,
      qtdVendidaTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[8]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(qtdVendidaTotal)

      qtdVendidaAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[8]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(qtdVendidaAGFNao)

      qtdVendidaAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[8]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(qtdVendidaAGFSim)

      #Tabela 9 = Variável 9,
      valorProducaoT = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[9]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(valorProducaoT)

      valorProducaoAGFN = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[9]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(valorProducaoAGFN)

      valorProducaoAGFS = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[9]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(valorProducaoAGFS)

      #Tabela 10 = Variável 10,
      valorVendaT = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[10]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(valorVendaT)

      valorVendaAGFN = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[10]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(valorVendaAGFN)

      valorVendaAGFS = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[10]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(valorVendaAGFS)

      #Tabela 11 = Variável 11,
      valorProducaoT = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[11]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(valorProducaoT)

      valorProducaoAGFN = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[11]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(valorProducaoAGFN)

      valorProducaoAGFS = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[11]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(valorProducaoAGFS)

      #Tabela 12 = Variável 12,
      valorVendaT = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[12]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(valorVendaT)

      valorVendaAGFN = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[12]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(valorVendaAGFN)

      valorVendaAGFS = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[12]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(valorVendaAGFS)

      for col in range(0, 5):
        info.append(cidades_array[estado][col])

    time.sleep(1)

  info_array = np.array(info).reshape(-1, 44)
  df = pd.DataFrame(info_array, columns=colunas_sidra_6956)
  with pd.option_context('display.max_rows', None,
      'display.max_columns', None):
    df.to_csv('info_br_fruticultura.csv', sep =';', index = False, encoding="utf-8")
    files.download('info_br_fruticultura.csv')

except OSError as e:

  print(e.args)
  print(type(e))
  print(e)
  print(e.with_traceback)

Rondônia
Acre
Amazonas
Roraima
Pará
Amapá
Tocantins
Maranhão
Piauí
Ceará
Rio Grande do Norte
Paraíba
Pernambuco
Alagoas
Sergipe
Bahia
Minas Gerais
Espírito Santo
Rio de Janeiro
São Paulo
Paraná
Santa Catarina
Rio Grande do Sul
Mato Grosso do Sul
Mato Grosso
Goiás
Distrito Federal


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Código de extração dos dados do CONAB

In [ ]:
try:

  colunas_conab = ['Sigla UF','Produto','Quantidade(kg)','Valor (R$)','Preço Médio (R$)']
  info = []
  linhas = 534
  driver.get("http://dw.ceasa.gov.br/")

  driver.implicitly_wait(15)

  botao = wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"run_query"))).click()
  driver.implicitly_wait(10)
  option = wait.until(EC.element_to_be_clickable(
      (By.XPATH,"//*[@id='tab_panel']/div/div[2]/div[2]/select/optgroup/option[2]")))
  option.click()
  driver.implicitly_wait(10)
  bQuantidade = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[1]/ul/li/ul/li[1]/a'))).click()
  bValor = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[1]/ul/li/ul/li[2]/a'))).click()
  bPreco = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[1]/ul/li/ul/li[3]/a'))).click()
  bProdutoLista = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[6]/a'))).click()
  bProduto = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[6]/ul/ul/li[2]/a'))).click()
  bUFCeasa = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[2]/a'))).click()
  bUF = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[2]/ul/ul/li[3]/a'))).click()
  bPeriodoComercializacao = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[4]/a'))).click()
  #bAno = wait.until(EC.element_to_be_clickable(
  #    (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[4]/ul/ul/li[2]/a'))).click()

  time.sleep(10)
  botao = driver.find_element(By.XPATH,'//*[@id="tab_panel"]/div/div[1]/div/div[2]/div/ul/li[2]/a').click()
  time.sleep(20)
  controle = 0

  #driver.save_screenshot('teste.png')
  #files.download('teste.png')

  for numeroUF in range(2, 14):
    uf = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="table_27"]/table/thead/tr[1]/th[%d]/div' % numeroUF))).text
    for linha in range(1, linhas):
      info.append(uf)
      print(uf)
      produto = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="table_27"]/table/tbody/tr[%d]/th/div' % linha))).text
      info.append(produto)
      for coluna in range(1, 4):
        xpath = '//*[@id="table_27"]/table/tbody/tr[%d]/td[%d]' % (linha, (coluna+controle))
        dado = wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
        if dado.text != '':
          info.append(dado.text)
        else:
          info.append('X')
    controle += 3
  estados = ['AP','AM','BA','MA','MT','MS','PA','PB','PI','RN','RO','RR','SC','SE','TO']
  for estado in estados:
    for produto in range(0, linhas):
      info.append(estado)
      info.append(info[1+(produto*5)])
      for i in range(1, 4):
        info.append('X')
  info_array = np.array(info).reshape(-1, 5)
  df = pd.DataFrame(info_array, columns=colunas_conab)
  with pd.option_context('display.max_rows', None,
      'display.max_columns', None):
    df.to_csv('conab.csv', sep =';', index = False, encoding="utf-8")
    files.download('conab.csv')

except OSError as e:

  print(e.args)
  print(type(e))
  print(e)
  print(e.with_traceback)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

A saída de streaming foi truncada nas últimas 5000 linhas.
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE
CE

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Código de Extração do Conab com filtro de ano



In [ ]:
try:

  colunas_conab = ['Sigla UF','Produto','Quantidade Vendida (kg) em 2018','Valor Comercialiazado (R$) em 2018','Preço Médio Praticado (R$) em 2018',
                   'Quantidade Vendida (kg) em 2019','Valor Comercializado (R$) em 2019','Preço Médio Praticado (R$) em 2019',
                   'Quantidade Vendida (kg) em 2020','Valor Comercializado (R$) em 2020', 'Preço Médio Praticado (R$) em 2020',
                   'Quantidade Vendida (kg) em 2021','Valor Comercializado (R$) em 2021','Preço Médio Praticado (R$) em 2021',
                   'Quantidade Vendida (kg) em 2022','Valor Comercializado (R$) em 2022','Preço Médio Praticado (R$) em 2022',
                   'Quantidade Vendida (kg) em 2023','Valor Comercializado (R$) em 2023','Preço Médio Praticado (R$) em 2023']
  info = []
  produtos_inicio = 1
  produtos_fim = 50
  quantidade_produtos = 50
  driver.get("http://dw.ceasa.gov.br/")

  driver.implicitly_wait(15)

  botao = wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"run_query"))).click()
  driver.implicitly_wait(10)
  option = wait.until(EC.element_to_be_clickable(
      (By.XPATH,"//*[@id='tab_panel']/div/div[2]/div[2]/select/optgroup/option[2]")))
  option.click()
  driver.implicitly_wait(10)
  bQuantidade = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[1]/ul/li/ul/li[1]/a'))).click()
  bValor = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[1]/ul/li/ul/li[2]/a'))).click()
  bPreco = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[1]/ul/li/ul/li[3]/a'))).click()
  bProdutoLista = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[6]/a'))).click()
  bProduto = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[6]/ul/ul/li[2]/a'))).click()
  bUFCeasa = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[2]/a'))).click()
  bUF = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[2]/ul/ul/li[3]/a'))).click()
  bPeriodoComercializacao = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[4]/a'))).click()
  bAno = wait.until(EC.element_to_be_clickable(
      (By.XPATH,'//*[@id="tab_panel"]/div/div[2]/div[3]/div/div[2]/ul/li[4]/ul/ul/li[2]/a'))).click()

  time.sleep(10)
  botao = driver.find_element(By.XPATH,'//*[@id="tab_panel"]/div/div[1]/div/div[2]/div/ul/li[2]/a').click()
  time.sleep(20)
  controle = 0

  #driver.save_screenshot('teste.png')
  #files.download('teste.png')

  for numeroUF in range(2, 14):
    print(controle)
    uf = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="table_27"]/table/thead/tr[1]/th[%d]/div' % numeroUF))).text
    print(uf)
    if numeroUF != 4 and numeroUF != 13:
      for linha in range(produtos_inicio, produtos_fim):
        info.append(uf)
        produto = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="table_27"]/table/tbody/tr[%d]/th/div' % linha))).text
        info.append(produto)
        for coluna in range(1, 19):
          xpath = '//*[@id="table_27"]/table/tbody/tr[%d]/td[%d]' % (linha, (coluna+controle))
          dado = wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
          if dado.text != '':
            info.append(dado.text)
          else:
            info.append('X')
      controle += 18
    else:
      if numeroUF == 4:
        for linha in range(produtos_inicio, produtos_fim):
          info.append(uf)
          produto = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="table_27"]/table/tbody/tr[%d]/th/div' % linha))).text
          info.append(produto)
          for coluna in range(1, 4):
            info.append('X')
          for coluna in range(1, 7):
            xpath = '//*[@id="table_27"]/table/tbody/tr[%d]/td[%d]' % (linha, (coluna+controle))
            dado = wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
            if dado.text != '':
              info.append(dado.text)
            else:
              info.append('X')
          for coluna in range(1, 10):
            info.append('X')
        controle += 6
      else:
        for linha in range(produtos_inicio, produtos_fim):
          info.append(uf)
          produto = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="table_27"]/table/tbody/tr[%d]/th/div' % linha))).text
          info.append(produto)
          for coluna in range(1, 16):
            xpath = '//*[@id="table_27"]/table/tbody/tr[%d]/td[%d]' % (linha, (coluna+controle))
            dado = wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
            if dado.text != '':
              info.append(dado.text)
            else:
              info.append('X')
          for coluna in range(1, 4):
            info.append('X')
          controle += 18
  estados = ['AP','AM','BA','MA','MT','MS','PA','PB','PI','RN','RO','RR','SC','SE','TO']
  for estado in estados:
    for produto in range(0, quantidade_produtos):
      info.append(estado)
      info.append(info[1+(produto*20)])
      for i in range(1, 19):
        info.append('X')
  info_array = np.array(info).reshape(-1, 20)
  df = pd.DataFrame(info_array, columns=colunas_conab)
  with pd.option_context('display.max_rows', None,
      'display.max_columns', None):
    df.to_csv('conab_completop1.csv', sep =';', index = False, encoding="utf-8")
    files.download('conab_completop1.csv')

except OSError as e:

  print(e.args)
  print(type(e))
  print(e)
  print(e.with_traceback)

0
DF
18
GO
36
AL
42
CE
60
PE
78
AC
96
ES
114
MG
132
RJ
150
SP
168
PR
186
RS


TimeoutException: ignored

Download dos CSV produzidos com os dados extraídos

In [ ]:
files.download('conab.csv')
files.download('cidades.csv')
files.download('info_br.csv')

Código de junção dos dados do SIDRA com os dados da população

In [ ]:
df_ibge = pd.read_csv('cidades.csv')
df_sidra = pd.read_csv('info_br.csv')

values_ibge = df_ibge.values
#print('Quantidade de elementos = %d' % values_ibge.size)

values_sidra = df_sidra.values
#print('Quantidade de elementos = %d' % values_sidra.size)

colunas_pop_prod = ['Produto','Nome UF','qtd Estabelecimentos','qtd Estabelecimentos AGFNão','qtd Estabelecimentos AGFSim','qtd Produzida',
              'qtd Produzida AGFNao','qtd Produzida AGFSim', 'Unidade de Medida', 'qtd Vendida','qtd Vendida AGFNao','qtd Vendida AGFSim',
              'R$ Produção Total','R$ Produção AGFNao','R$ Produção AGFSim','R$ Venda Total','R$ Venda AGFNao','R$ Venda AGFSim',
              'Sigla UF', 'Nome UF' ,'População Total em 2010','População Urbana em 2010','População Rural em 2010']

linhas_pop_prod = []

for linha_ibge in values_ibge:
  inf_ibge = str(linha_ibge).split(';')
  for linha_sidra in values_sidra:
    inf_sidra = str(linha_sidra).split(';')
    distance = Levenshtein.distance(inf_ibge[1], inf_sidra[1])
    if distance == 0:
      #mudar os nomes das colunas
      inf_sidra[0] = inf_sidra[0].replace("['",'')
      inf_sidra[17] = inf_sidra[17].replace("']","")
      inf_ibge[0] = inf_ibge[0].replace("['",'')
      inf_ibge[4] = inf_ibge[4].replace("']","")
      linhas_pop_prod.append(inf_sidra+inf_ibge)

df_pop_prod = pd.DataFrame(linhas_pop_prod,columns=colunas_pop_prod)

with pd.option_context('display.max_rows', None,
      'display.max_columns', None):
    df_pop_prod.to_csv('info_populacao_producao.csv', sep =';', index = False, encoding="utf-8")
    files.download('info_populacao_producao.csv')

FileNotFoundError: ignored

Código de junção dos demais dados com os dados do CONAB

In [ ]:
df_parcial = pd.read_csv('info_populacao_producao.csv', sep=';')
df_conab = pd.read_csv('conab.csv', sep=';')

values_parcial = df_parcial.values
#print('Quantidade de elementos = %d' % values_parcial.size)

values_conab = df_conab.values
#print('Quantidade de elementos = %d' % values_conab.size)

colunas_completo = colunas_pop_prod+colunas_conab

linhas_completo = []

for linha_conab in values_conab:
  for linha_sidra in values_parcial:
    aux_prod_conab = linha_conab[1].upper()
    aux_prod_parcial = linha_parcial[0].upper()
    distance_sigla = Levenshtein.distance(linha_parcial[18], linha_conab[0])
    distance_produtos = Levenshtein.distance(aux_prod_parcial, aux_prod_conab)
    if distance_sigla == 0 and distance_produtos < 2:
      #mudar os nomes das colunas
      linha_parcial[0] = linha_parcial[0].replace("['",'')
      linha_parcial[22] = str(linha_parcial[22]).replace("']","")
      linha_conab[0] = linha_conab[0].replace("['",'')
      linha_conab[4] = str(linha_conab[4]).replace("']","")
      linhas_completo.append(str(linha_parcial)+str(linha_conab))

df_completo = pd.DataFrame(linhas_completo,columns=colunas_completo)

with pd.option_context('display.max_rows', None,
      'display.max_columns', None):
    df_completo.to_csv('dataset_completo.csv', sep =';', index = False, encoding="utf-8")
    files.download('dataset_completo.csv')

A saída de streaming foi truncada nas últimas 5000 linhas.
PE
GYPSOFILA
CENOURA
MG
PE
HELIANTUS
CENOURA
MG
PE
HELICONIA
CENOURA
MG
PE
HORENCO
CENOURA
MG
PE
HORTELA
CENOURA
MG
PE
HORTENCIA
CENOURA
MG
PE
IMPATINS
CENOURA
MG
PE
INGA
CENOURA
MG
PE
INHAME
CENOURA
MG
PE
IOGURTE
CENOURA
MG
PE
IRIS
CENOURA
MG
PE
ITENS DIVERSOS
CENOURA
MG
PE
JABUTICABA
CENOURA
MG
PE
JACA
CENOURA
MG
PE
JAMBO
CENOURA
MG
PE
JAMBU
CENOURA
MG
PE
JAMELAO
CENOURA
MG
PE
JANAUBA
CENOURA
MG
PE
JASMIM
CENOURA
MG
PE
JATOBA
CENOURA
MG
PE
JAU
CENOURA
MG
PE
JENIPAPO
CENOURA
MG
PE
JILO
CENOURA
MG
PE
JUNDIA
CENOURA
MG
PE
JURUBEBA
CENOURA
MG
PE
KALANCHOE
CENOURA
MG
PE
KANI KAMA
CENOURA
MG
PE
KINO
CENOURA
MG
PE
KIWI
CENOURA
MG
PE
LACRAIA
CENOURA
MG
PE
LAGOSTA
CENOURA
MG
PE
LAMBARI
CENOURA
MG
PE
LARANJA
CENOURA
MG
PE
LEITE
CENOURA
MG
PE
LENKON
CENOURA
MG
PE
LENTILHA
CENOURA
MG
PE
LESTE
CENOURA
MG
PE
LICHIA
CENOURA
MG
PE
LIMA DA PERSIA
CENOURA
MG
PE
LIMAO
CENOURA
MG
PE
LINGUADO
CENOURA
MG
PE
LINGUICA
CENOURA
MG
PE
LIRIO
CENOURA
MG


Junção do Dataset Completo pelo carregamento do CSV pelo drive

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')
df_parcial = pd.read_csv('/content/drive/MyDrive/dataset/info_populacao_producao.csv', ';', encoding="utf-8")
df_conab = pd.read_csv('/content/drive/MyDrive/dataset/conab.csv', ';', encoding="utf-8")

values_parcial = df_parcial.values
#print('Quantidade de elementos = %d' % values_parcial.size)

values_conab = df_conab.values
#print('Quantidade de elementos = %d' % values_conab.size)

colunas_completo = ['Produto','Nome UF','qtd Estabelecimentos','qtd Estabelecimentos AGFNão','qtd Estabelecimentos AGFSim','qtd Produzida',
              'qtd Produzida AGFNao','qtd Produzida AGFSim', 'Unidade de Medida', 'qtd Vendida','qtd Vendida AGFNao','qtd Vendida AGFSim',
              'R$ Produção Total','R$ Produção AGFNao','R$ Produção AGFSim','R$ Venda Total','R$ Venda AGFNao','R$ Venda AGFSim',
              'Sigla UF', 'Nome UF' ,'População Total em 2010','População Urbana em 2010','População Rural em 2010',
              'Sigla UF','Produto','Quantidade(kg)','Valor (R$)','Preço Médio (R$)']

linhas_completo = []

for linha_conab in values_conab:
  for linha_parcial in values_parcial:
    aux_prod_conab = linha_conab[1]
    aux_prod_parcial = linha_parcial[0]
    distance_sigla = Levenshtein.distance(linha_parcial[18], linha_conab[0])
    distance_produtos = Levenshtein.distance(aux_prod_parcial.upper(), aux_prod_conab.upper())
    if distance_sigla == 0 and distance_produtos < 2:
      linha_parcial[0] = linha_parcial[0].replace("['",'')
      linha_parcial[22] = str(linha_parcial[22]).replace("']","")
      linha_conab[0] = linha_conab[0].replace("['",'')
      linha_conab[4] = str(linha_conab[4]).replace("']","")
      linhas_completo.append(np.concatenate((np.array(linha_parcial),np.array(linha_conab))))
      break

df_completo = pd.DataFrame(linhas_completo,columns=colunas_completo)
indices_para_remover = [19, 23, 24]
df_completo = df_completo.iloc[:, [col for col in range(len(df_completo.columns)) if col not in indices_para_remover]]

with pd.option_context('display.max_rows', None,
      'display.max_columns', None):
    df_completo.to_csv('dataset_completo.csv', sep =';', index = False, encoding="utf-8")
    files.download('dataset_completo.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-40-c969bc862666>:5: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_parcial = pd.read_csv('/content/drive/MyDrive/dataset/info_populacao_producao.csv', ';', encoding="utf-8")
<ipython-input-40-c969bc862666>:6: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_conab = pd.read_csv('/content/drive/MyDrive/dataset/conab.csv', ';', encoding="utf-8")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Exemplo da Resolução de problema da configuração

In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Executar o Chrome em modo headless (sem interface gráfica)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
service = Service('/usr/lib/chromedriver')
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://www.example.com')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Buildin

Código Completo de Extração de todas os sites

In [ ]:
!pip install python-Levenshtein
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!apt-get update
!google-chrome-stable --version
!apt -y update
!apt install -y wget curl unzip
!wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
!dpkg -i libu2f-udev_1.1.4-1_all.deb
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
!wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
!unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver
!mv /tmp/chromedriver /usr/lib/chromedriver

import time
from selenium import webdriver
from time import sleep
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import pandas as pd
import numpy as np
import os
from google.colab import drive
from google.colab import files
import Levenshtein
from selenium.webdriver.chrome.service import Service

try:

  str1 = "gato"
  str2 = "cachorro"
  distance = Levenshtein.distance(str1, str2)
  print("Distância entre as palavras %d" % distance)

  options = webdriver.ChromeOptions()
  options.add_argument('--headless')  # Executar o Chrome em modo headless (sem interface gráfica)
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  service = Service('/usr/lib/chromedriver')
  driver = webdriver.Chrome(service=service, options=options)

  #drive.mount('/content/drive')
  wait = WebDriverWait(driver, 20)
  cidades = []
  colunas = ['Sigla UF', 'Nome UF','População Total em 2010','População Urbana em 2010','População Rural em 2010']

  site = "https://www.ibge.gov.br/censo2010/apps/sinopse/index.php"
  driver.get(site)
  driver.implicitly_wait(20)
  siglas = ['RO','AC','AM','RR','PA','AP','TO','MA','PI','CE','RN','PB','PE','AL','SE','BA','MG','ES','RJ','SP','PR','SC','RS','MS','MT','GO','DF']

  opcaoUF = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/form/select[1]/option[2]'))).click()
  time.sleep(2)
  opcaoDados = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/form/select[2]/option[12]'))).click()
  time.sleep(2)
  for i in range(1,28):
    #//*[@id="div_tabela_dados"]/table/tbody/tr[27]/td[2]
    #2 nomeuf
    #3 Pop total
    #4 total homens
    #5 total mulheres
    #6 total urbana
    #7 urbana homens
    #8 urbana mulheres
    #9 rural total
    #10 rural homens
    #11 rural mulheres

    cidades.append(siglas[i-1])
    nomeuf = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="div_tabela_dados"]/table/tbody/tr[%d]/td[2]' % i))).text
    cidades.append(nomeuf)

    populacaototal = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="div_tabela_dados"]/table/tbody/tr[%d]/td[3]' % i))).text
    populacaototal = populacaototal.replace('.','')
    cidades.append(populacaototal)

    populacaourbana = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="div_tabela_dados"]/table/tbody/tr[%d]/td[6]' % i))).text
    populacaourbana = populacaourbana.replace('.','')
    cidades.append(populacaourbana)

    populacaorural = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="div_tabela_dados"]/table/tbody/tr[%d]/td[9]' % i))).text
    populacaorural = populacaorural.replace('.','')
    cidades.append(populacaorural)

  cidades_array = np.array(cidades).reshape(-1, 5)
  df = pd.DataFrame(cidades_array, columns=colunas)
  with pd.option_context('display.max_rows', None,
      'display.max_columns', None):
    df.to_csv('cidades.csv', sep =';', index = False, encoding="utf-8")
    #files.download('cidades.csv')

  for estado in range(0, 27):

    driver.get("https://sidra.ibge.gov.br/tabela/6954")
    driver.implicitly_wait(10)
    wait = WebDriverWait(driver, 20)
    #Origem: //*[@id="dim-C220"] bloco grupo de área total
    origem = wait.until(EC.visibility_of_element_located((By.ID, 'dim-C220')))
    #Destino: //*[@id="dim-V"] bloco variável
    destino = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="editor-layout"]/tbody/tr[1]/th')))
    acao = ActionChains(driver)
    time.sleep(1)
    #Grupo de área total
    acao.drag_and_drop(origem, destino).perform()
    time.sleep(1)
    #Origem: //*[@id="dim-C228"] Produtos
    origem = wait.until(EC.visibility_of_element_located((By.ID, 'dim-C228')))
    destino = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="editor-layout"]/tbody/tr[7]')))
    acao = ActionChains(driver)
    time.sleep(2)
    #Produtos
    acao.drag_and_drop(origem, destino).perform()
    time.sleep(1)
    #Origem: //*[@id="dim-T"] Unidade territorial
    origem = wait.until(EC.visibility_of_element_located((By.ID, 'dim-T')))
    #Destino: //*[@id="dim-V"] bloco variável
    destino = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="editor-layout"]/tbody/tr[1]/th')))
    acao = ActionChains(driver)
    time.sleep(1)
    #Unidade territorial
    acao.drag_and_drop(origem, destino).perform()
    time.sleep(1)

    if estado == 0:
      #Marca os dois botões de Tipologia
      tipAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="panel-C829-collapse"]/div[2]/div/div[2]/div/div/div/div/div[2]/div/div/div/button'))).click()
      tipAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="panel-C829-collapse"]/div[2]/div/div[2]/div/div/div/div/div[3]/div/div/div/button'))).click()
      #Desmarca o Brasil
      buttonBrasil = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="arvore-1025e-1"]/div/div/div/button'))).click()

    #Marcar todas as variáveis
    buttonVariaveis = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="panel-V-collapse"]/div[2]/div/div[1]/div[1]/div/button[1]'))).click()

    #Marcar todos os produtos
    buttonProdutos = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="panel-C228-collapse"]/div[2]/div/div[1]/div[1]/div/button[1]'))).click()

    colunas = ['Produto','Local','qtd Estabelecimentos','qtd Estabelecimentos AGFNão','qtd Estabelecimentos AGFSim','qtd Produzida',
              'qtd Produzida AGFNao','qtd Produzida AGFSim', 'Unidade de Medida', 'qtd Vendida','qtd Vendida AGFNao','qtd Vendida AGFSim',
              'R$ Produção Total','R$ Produção AGFNao','R$ Produção AGFSim','R$ Venda Total','R$ Venda AGFNao','R$ Venda AGFSim']
    info = []

    #Constantes
    quantidade_produtos = 57
    quantidade_total = quantidade_produtos*19

    #Clica no botão para exibir os municípios
    botaoEstado = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="arvore-1105e-1"]/div/div'))).click()

    botaoDesmarcaEstado = wait.until(EC.element_to_be_clickable(
        (By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[1]/div[1]/div/button[2]'))).click()

    a = ActionChains(driver)
    iconeLupa = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[1]/div[2]/div[1]/span')))
    a.move_to_element(iconeLupa).perform()
    time.sleep(2)

    #Campo de inserção do nome do munícipio
    inputNomeEstado = wait.until(EC.visibility_of_element_located((
        By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[1]/div[2]/div[2]/div/input'
    )))
    inputNomeEstado.send_keys(cidades_array[estado][1])
    print(cidades_array[estado][1])
    time.sleep(3)
    #//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/div[1]/div/div/div/button

    buttonEstado = wait.until(EC.visibility_of_element_located((
        By.XPATH, '//*[@id="linha-territorios"]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/div[1]/div/div/div/button'
    )))
    buttonEstado.click()

    time.sleep(2)

    buttonVisualizar = wait.until(EC.element_to_be_clickable((By.ID, 'botaoOk'))).click()
    time.sleep(6)

    #driver.save_screenshot('teste.png')
    #files.download('teste.png')

    for i in range(2, quantidade_produtos+2):

      produto = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/th/span' % i))).text
      info.append(produto)

      info.append(cidades_array[estado][1])

      #Tabela 1 = Variável 1, Número de estabelecimentos
      numEstabelecimentoTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(numEstabelecimentoTotal)

      numEstabelecimentoAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(numEstabelecimentoAGFNao)

      numEstabelecimentoAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[1]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(numEstabelecimentoAGFSim)

      #Tabela 2 = Variável 2, Quantidade Produzida
      qtdProduzidaTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(qtdProduzidaTotal)

      qtdProduzidaAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(qtdProduzidaAGFNao)

      qtdProduzidaAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(qtdProduzidaAGFSim)

      #Unidade de Medida
      unidade = wait.until(EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[2]/table/tbody/tr[%d]/th/span/span' % (i+1)))).text
      unidade = unidade.replace("(","")
      unidade = unidade.replace(")","")
      info.append(unidade)

      #Tabela 3 = Variável 3, Quantidade Vendida
      qtdVendidaTotal = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[3]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(qtdVendidaTotal)

      qtdVendidaAGFNao = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[3]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(qtdVendidaAGFNao)

      qtdVendidaAGFSim = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[3]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(qtdVendidaAGFSim)

      #Tabela 4 = Variável 4, Valor Produção
      valorProducaoT = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[4]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(valorProducaoT)

      valorProducaoAGFN = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[4]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(valorProducaoAGFN)

      valorProducaoAGFS = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[4]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(valorProducaoAGFS)

      #Tabela 5 = Variável 5, Valor Venda
      valorVendaT = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[5]/table/tbody/tr[%d]/td[1]' % i))).text
      info.append(valorVendaT)

      valorVendaAGFN = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[5]/table/tbody/tr[%d]/td[2]' % i))).text
      info.append(valorVendaAGFN)

      valorVendaAGFS = wait.until(EC.visibility_of_element_located(
          (By.XPATH, '//*[@id="resultado"]/div/div[3]/div/div/div/div[5]/table/tbody/tr[%d]/td[3]' % i))).text
      info.append(valorVendaAGFS)

    time.sleep(1)

  info_array = np.array(info).reshape(-1, 18)
  df = pd.DataFrame(info_array, columns=colunas)
  with pd.option_context('display.max_rows', None,
      'display.max_columns', None):
    df.to_csv('info_br.csv', sep =';', index = False, encoding="utf-8")
    files.download('info_br.csv')

except OSError as e:

  print(e.args)
  print(type(e))
  print(e)
  print(e.with_traceback)